In [27]:
from gen_err import SynthesizeData
from model import SpellCorrectionModel
import torch
import torch.nn as nn
import math
from torch import Tensor
import traceback
import torch.optim as optim
from tqdm.notebook import tqdm
import string
import ast
import pandas as pd
import copy

torch.autograd.set_detect_anomaly(True)

In [28]:
with open('merge_word_vocab_13126.txt','r') as f:
    word_tokenizer = f.read().split('\n')
word_tokenizer.insert(0,"<unk>")
word_tokenizer.insert(0,"<pad>")
# word_tokenizer.insert(0,"<end>")
# word_tokenizer.insert(0,"<start>")

map_word = {j:i for i,j in enumerate(word_tokenizer)}
reverse_map_word = {map_word[i]:i for i in map_word.keys()}
# map_word

In [29]:
with open('char_vocab.txt','r') as f:
    char_tokenizer = f.read().split('\n')
char_tokenizer.insert(0,"<unk>")
# char_tokenizer.insert(0,"<end>")
# char_tokenizer.insert(0,"<start>")
char_tokenizer.insert(0,"<pad>")

map_char = {j:i for i,j in enumerate(char_tokenizer)}
reverse_map_char = {map_char[i]:i for i in map_char.keys()}
# map_char

In [30]:
def mapping_batch(sample):
    try:
        max_word_length = max([len(x.split(' ')) for x in sample])
        max_char_length = max([max([len(word) for word in sentence.split(' ')]) for sentence in sample])
    except:
        return "", "", 0
    # #print(max_word_length, max_char_length)
    res = []
    res_char = []
    for i in sample:
        out = []
        out_char = []
        for j in i.split(' '):
            out.append(map_word[j.lower()] if j.lower() in map_word.keys() else map_word['<unk>'])
            r = []
            for g in j:
                r.append(map_char[g] if g in map_char.keys() else map_char['<unk>'])
            while len(r) < max_char_length:
                r.append(0)
            out_char.append(r)
        while len(out) < max_word_length:
            out.append(0)
            out_char.append([0] * max_char_length)
            
        res.append(out)
        res_char.append(out_char)
    return res, res_char , max_word_length

# mapping_batch(sample_text)

In [31]:
sample_text = ["Chây ì nộp phạt nguội"]
mapping_batch(sample_text)

([[3581, 3364, 965, 729, 2622]],
 [[[21, 126, 103, 174, 0],
   [128, 0, 0, 0, 0],
   [136, 148, 155, 0, 0],
   [155, 126, 96, 159, 0],
   [136, 125, 160, 148, 127]]],
 5)

In [32]:
# with open('Viet74K.txt','r') as f:
#     data = f.read().split()
# data = [x.lower().split(' ')[0] for x in data]
# data = list(set(data))
# len(data)

In [33]:
# with open('vietnamese.txt','w') as f:
#     for i in data:
#         f.write(i)
#         f.write('\n')

In [34]:
gen_spell = SynthesizeData("vietnamese.txt")

In [35]:
res = gen_spell.add_noise("Đây cũng là trận đấu chính thức đầu tiên của huấn luyện viên Kiatisak sau khi đảm nhận cương vị huấn luyện viên trưởng của Công an Hà Nội. ", 0.2)
for i,j in zip(res[0].split(' '), res[1]):
    print(i, j)

Đây 0
cũng 0
là 0
trận 0
đấu 0
chính 0
thức 0
đầu 0
tiên 0
của 0
huấn 0
luyện 0
viên 0
Kiatisak 0
sau 0
khi 0
đảm 0
nhận 0
cuwơng 1
vị 0
huấn 0
luyện 0
viên 0
trưởng 0
của 0
Công 0
an 0
Hxà 1
Nội 0
. 0


In [36]:
s = """
aàáảãạ
ăằắẳẵặ
âầấẩẫậ
bc
dđ
eèéẻẽẹ
êềếểễệ
gh
iìíỉĩị
lkmn
oòóỏõọ
ôồốổỗộ
ơờớởỡợ
pqrst
uùúủũụ
ưừứửữự
vx
yỳýỷỹỵ
"""
all_char = []
for i in s:
    if i == '\n':
        continue
    all_char.append(i)

In [37]:
all_char

['a',
 'à',
 'á',
 'ả',
 'ã',
 'ạ',
 'ă',
 'ằ',
 'ắ',
 'ẳ',
 'ẵ',
 'ặ',
 'â',
 'ầ',
 'ấ',
 'ẩ',
 'ẫ',
 'ậ',
 'b',
 'c',
 'd',
 'đ',
 'e',
 'è',
 'é',
 'ẻ',
 'ẽ',
 'ẹ',
 'ê',
 'ề',
 'ế',
 'ể',
 'ễ',
 'ệ',
 'g',
 'h',
 'i',
 'ì',
 'í',
 'ỉ',
 'ĩ',
 'ị',
 'l',
 'k',
 'm',
 'n',
 'o',
 'ò',
 'ó',
 'ỏ',
 'õ',
 'ọ',
 'ô',
 'ồ',
 'ố',
 'ổ',
 'ỗ',
 'ộ',
 'ơ',
 'ờ',
 'ớ',
 'ở',
 'ỡ',
 'ợ',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'ù',
 'ú',
 'ủ',
 'ũ',
 'ụ',
 'ư',
 'ừ',
 'ứ',
 'ử',
 'ữ',
 'ự',
 'v',
 'x',
 'y',
 'ỳ',
 'ý',
 'ỷ',
 'ỹ',
 'ỵ']

In [38]:
import re
import string


def split_punc_to_word2(sentence):
    separated_sentence = sentence.replace("'",'"')
    separated_sentence = re.sub(r"([\w/'+$\s-]+|[^\w/'+$\s-]+)\s*", r"\1 ", sentence)
    # print(separated_sentence)
    separated_sentence = " ".join(separated_sentence.split())
    return separated_sentence


def split_punc_to_word(sentence):
    tmp = sentence.split(' ')
    res = []
    for i in tmp:
        r, l = -1, len(i) + 1
        for idx,char in enumerate(i):
            if char.isalpha() and char.lower() in all_char:
                if r == -1:
                    r = idx
                l = idx
        if l == len(i) + 1:
            l = len(i)
        res.append(" ".join((i[:r] + " " + i[r:l+1] + " " + i[l+1:]).split()))
    out = " ".join(x for x in res)
    return split_punc_to_word2(out)
                    
            


In [39]:
s = "Hàng chục ngàn), phuong tiện bỵ ghi hình vi phạm luật gizo thông ở P . HCM , bị 'bêu tên ' nhưng chủ vẫn khôn chịu nộp phạt ."
split_punc_to_word(s)

"Hàng chục ngàn ), phuong tiện bỵ ghi hình vi phạm luật gizo thông ở P . HCM , bị ' bêu tên ' nhưng chủ vẫn khôn chịu nộp phạt ."

In [40]:
# # Gen training data

# from tqdm.notebook import tqdm
# import pandas as pd
# cnt = 0
# lst_gt = []
# lst_gen = []
# spell_label = []
# cap_label = []
# with open('corpus-full.txt', 'r') as file:
#     for line in tqdm(file, total=cnt):
#         cnt += 1
#         s = []
#         line = split_punc_to_word(line)
#         # print(line)
#         res = gen_spell.add_noise(line)
#         # print(res)
#         lst_gt.append(line)
#         lst_gen.append(res[0])
#         label_cap = [1 if x[0].isupper() else 0 for x in line.split(' ')]
#         label_spell = [0 if x != 1 else 1 for x in res[1]]
#         spell_label.append(label_spell)
#         cap_label.append(label_cap)
#         if cnt == 1000000:
#             break

# df = pd.DataFrame({"text":lst_gt,'generate':lst_gen,"spell_label":spell_label,"cap_label":cap_label})
# # print(df)
# df.to_csv('train_format.csv',index=False)

In [16]:
# Define your custom loss function
import torch.nn as nn
import torch.nn.functional as F
import torch 

device = "cuda" if torch.cuda.is_available() else "cpu" 

# def custom_focal_loss(output, target):
#     alpha = 0.25  # balancing parameter
#     gamma = 2.0   # focusing parameter
#     bce_loss = bce_l(output, target)
#     pt = torch.exp(-bce_loss)
#     focal_loss = alpha * (1 - pt) ** gamma * bce_loss
#     return focal_loss

class CustomLoss(nn.Module):
    def __init__(self, weight = [1,1,1]):
        super(CustomLoss, self).__init__()
        self.weight = weight

    def forward(self, output_correction, output_spell, output_upper, target, total_length):
        for k in target.keys():
            if k == 'correction':
                target[k] = target[k].to(device)
            else:
                target[k] = target[k].to(device).float()
        # print(target_correction.shape, output_correction.shape, output_spell.shape, target_spell.shape)
        # Calculate the loss for is_correct prediction
        # bce_loss_upper = nn.BCEWithLogitsLoss(reduction='mean', pos_weight=(target['upper']*3).view(-1, 1)).to(device)
        # bce_loss_spell = nn.BCEWithLogitsLoss(reduction='mean', pos_weight=(target['spell']*3).view(-1, 1)).to(device)
        output_spell = output_spell.float()
        # print(output_spell)
        output_upper = output_upper.float()
        # print(output_spell.shape, output_upper.shape)
        # print(target['spell'].shape, target['upper'].shape)
        # Calculate the loss for correction prediction
        mask = (target['spell'] != 0.0).float()

        # print("Output spell ",output_spell)
        
        # Calculate cross-entropy loss with the mask

        # print(output_correction.shape, mask.shape)

        output_correction = torch.permute(output_correction, (0,2,1))
        loss = F.cross_entropy(output_correction, target['correction'], reduction='none')  # 'none' to prevent reduction
        # print("loss ",loss.shape)
        # Apply the mask to ignore certain positions
        masked_loss = loss * mask
        
        # Calculate the mean loss ignoring certain positions
        correction_loss = torch.sum(masked_loss) / (torch.sum(mask)+ 1e-6)
        # print("corr ",correction_loss)
        # print("corr loss after mask ",correction_loss)
        is_correct_loss = bce_l(output_spell.view(-1, 1), target['spell'].view(-1, 1)) / total_length
        # print("is_correct_loss loss ",is_correct_loss)
        is_upper_loss = bce_l(output_upper.view(-1, 1), target['upper'].view(-1, 1)) / total_length
        # print("is_upper_loss loss ",is_upper_loss)
        # print(is_correct_loss, correction_loss, is_upper_loss)
        # Combine the two losses
        total_loss = self.weight[0] * correction_loss + self.weight[1] * is_correct_loss + self.weight[2] * is_upper_loss

        return total_loss, {"correct_loss":correction_loss.detach().cpu().item(),"spell_loss":is_correct_loss.detach().cpu().item(),"upper_loss":is_upper_loss.detach().cpu().item()}

# Create an instance of your model
spell_model = SpellCorrectionModel(13130,214).to(device)

# Define your optimizer
optimizer = optim.Adam(spell_model.parameters(), lr=1.76e-4)

# Define your custom loss function
loss_function = CustomLoss()
bce_l = nn.BCELoss(reduction='sum').to(device)

# Define your training loop
def train(model, optimizer, loss_function, word_text, char_text, target):
    model.train()

    # Clear the gradients
    optimizer.zero_grad()

    # Forward pass
    output_correction, output_spell, output_upper, total_length = model(word_text, char_text)

    #print(output_spell)

    # print("Model output shape : ",output_correction.shape, output_spell.shape)

    # Calculate the loss
    loss, detail = loss_function(output_correction, output_spell, output_upper, target, total_length)

    # Backward pass
    loss.backward()


    # for name, param in model.named_parameters():
    #     print(param.requires_grad)
    #     if param.grad is not None:
    #         print(f'Parameter: {name}, Gradient Norm: {param.grad.norm().item()}')
    #     else:
    #         print(f'Parameter: {name}, Gradient: None')

    # Update the weights
    optimizer.step()

    return loss.item(), detail

num_epochs = 1
batch_size = 8

In [41]:
df = pd.read_csv("train_format.csv")
df['word_count'] = df['text'].apply(lambda x : len(x.split(' ')))
df = df[(df['text'].apply(len) > 10)]
df = df[df['word_count'] <= 500]
df.head()

,text,generate,spell_label,cap_label,word_count
0,Chây ì nộp phạt nguội .,Chây ì nộp phạt ngui .,"[0, 0, 0, 0, 1, 0]","[1, 0, 0, 0, 0, 0]",6
1,Hàng chục ngàn phương tiện bị ghi hình vi phạm...,Hàng chxục ngàn phương tiện bị ghi hình vo phạ...,"[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",31
2,Trên cổng thông tin điện tử của Công an TP . H...,Trên cong thông tin điện tử ca Công ắn YP . HC...,"[0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",74
3,Đây là các phương tiện vi phạm được camera ( d...,Đây lờ các phương tiện vi phạm được camera ( d...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",23
4,"Điều đáng nói , dù Phòng CSGT đường bộ - đường...","Điều đáng nói , dù Pho2ng CSGT đường bộ - đườn...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, ...",49


In [42]:
def expand_to_max_length(x, max_l = 10):
    if isinstance(x, str):
        x = ast.literal_eval(x)
    while len(x) < max_l:
        x.append(0)
    return x

def get_correct(text, generate):
    text = text.split(' ')
    generate = generate.split(' ')
    res = []
    for i,j in zip(text, generate):
        if i != j:
            res.append(map_word[i.lower() if i.lower() in map_word.keys() else "<unk>"])
        else:
            res.append(1)
    return res

In [18]:
map_word['chây']

3581

In [38]:
batch_size = 16
num_epochs = 1
for epoch in range(num_epochs):
    print("EPOCH : {}".format(epoch))
    t_loss = []
    batch_detail = {"correct_loss":[],"spell_loss":[],"upper_loss":[]}
    for i in tqdm(range(0, df.shape[0]//batch_size, 1)):
        # Get the current batch
        tmp_data = df.iloc[batch_size * i: batch_size * (i+1)].to_dict('records')
        # print(tmp_data)

        batch_input_gt = [x['text'] for x in tmp_data]
        batch_input = [x['generate'] for x in tmp_data]

        # print(batch_input, batch_input_gt)

        t, c, max_batch_len = mapping_batch(batch_input)
        if max_batch_len == 0:
            continue
        # print(max_batch_len)
        label_spell = [expand_to_max_length(x['spell_label'], max_batch_len) for x in tmp_data]
        label_cap = [expand_to_max_length(x['cap_label'], max_batch_len) for x in tmp_data]
        label_correction = [get_correct(x['text'], x['generate']) for x in tmp_data]
        label_correction = [expand_to_max_length(x, max_batch_len) for x in label_correction]
        target = {
            "spell" : torch.tensor(label_spell),
            "upper" : torch.tensor(label_cap),
            "correction" : torch.tensor(label_correction)
        }
        
        # break
        try:
            # if t_correction.shape != torch.tensor(t).shape:
            #     continue
            loss, detail = train(spell_model, optimizer, loss_function, t, c, target)
            t_loss.append(loss)
            for detail_k in detail.keys():
                batch_detail[detail_k].append(detail[detail_k])
                
        except:
            # continue
            print(torch.tensor(label_spell).shape)
            print(torch.tensor(label_cap).shape)
            print(torch.tensor(label_correction).shape)
            print(traceback.format_exc())
            # print(batch_train_label.shape, t_correction.shape)
        if i % 1000 == 0:
            print(f"Epoch {epoch+1} - Batch {i} - Avg Loss: {(sum(t_loss)/len(t_loss)):.4f}")
            detail = {bdt: sum(batch_detail[bdt])/(len(batch_detail[bdt])) for bdt in batch_detail.keys()}
            print(detail)
            batch_detail = {"correct_loss":[],"spell_loss":[],"upper_loss":[]}
            t_loss = []
            if i % 2000 == 0 and i > 1:
                torch.save({
                    'model_state_dict': spell_model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                }, f'model_epochs_{epoch+1}_batch_{i}.pth')

EPOCH : 0


  0%|          | 0/62153 [00:00<?, ?it/s]

Epoch 1 - Batch 0 - Avg Loss: 2.6573
{'correct_loss': 2.5494680404663086, 'spell_loss': 0.06143359839916229, 'upper_loss': 0.04636283591389656}
Epoch 1 - Batch 1000 - Avg Loss: 1.4071
{'correct_loss': 1.3033928088247777, 'spell_loss': 0.06891965879499912, 'upper_loss': 0.034803896919474935}
Epoch 1 - Batch 2000 - Avg Loss: 1.5773
{'correct_loss': 1.4651899276077747, 'spell_loss': 0.08093883001804351, 'upper_loss': 0.031191074563510484}
Epoch 1 - Batch 3000 - Avg Loss: 1.5628
{'correct_loss': 1.4503035054206848, 'spell_loss': 0.0758976485952735, 'upper_loss': 0.036570466299832335}
Epoch 1 - Batch 4000 - Avg Loss: 1.4517
{'correct_loss': 1.3393171309828757, 'spell_loss': 0.07679312691837549, 'upper_loss': 0.03563258118071826}
Epoch 1 - Batch 5000 - Avg Loss: 1.4339
{'correct_loss': 1.3225603160113095, 'spell_loss': 0.07749408227391541, 'upper_loss': 0.033873894604155796}
Epoch 1 - Batch 6000 - Avg Loss: 1.5037
{'correct_loss': 1.3930859425365925, 'spell_loss': 0.07949477744475007, 'upper


KeyboardInterrupt



In [39]:
torch.save({
    'model_state_dict': spell_model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, f'model_latest.pth')

In [2]:
import torch
import torch.nn as nn
bce_l = nn.BCELoss(reduction='sum')
bce_l(torch.tensor([0.0,0.0]),torch.tensor([0.0,0.0]))

tensor(0.)

In [20]:
df.iloc[-1]

text           Trong dài hạn , Siegel cho biết : Những mối qu...
generate       Trong dài hạn , Siegel cho biết : Như4ng mối q...
spell_label    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, ...
cap_label      [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...
word_count                                                    21
Name: 999999, dtype: object

In [21]:
df.iloc[-1]['spell_label']

'[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]'

In [40]:
w, c, max_length = mapping_batch([df['generate'].iloc[0],df['generate'].iloc[-1]])

In [41]:
w

[[3581, 3364, 965, 729, 3, 13112, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [11,
  418,
  461,
  13110,
  3,
  12,
  80,
  13114,
  3,
  812,
  59,
  261,
  24,
  13,
  3,
  60,
  3,
  29,
  552,
  1201,
  13112]]

In [42]:
spell_model.eval()
with torch.no_grad():
    a, b, c, d = spell_model(w,c)

/home/ubuntu/miniconda3/lib/python3.11/site-packages/torch/nn/modules/transformer.py:562: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._transformer_encoder_layer_fwd(


In [43]:
d

27.0

In [44]:
torch.sigmoid(b)

tensor([[0.5017, 0.5006, 0.5002, 0.5006, 0.6974, 0.5000, 0.5000, 0.5000, 0.5000,
         0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
         0.5000, 0.5000, 0.5000],
        [0.5002, 0.5045, 0.5026, 0.5000, 0.5027, 0.5009, 0.5004, 0.5003, 0.7283,
         0.5020, 0.5010, 0.5003, 0.5011, 0.5001, 0.6719, 0.5002, 0.7303, 0.5025,
         0.5030, 0.5023, 0.5000]], device='cuda:0')

In [49]:
torch.sigmoid(c)

tensor([[0.7310, 0.5006, 0.5000, 0.5000, 0.5000, 0.5001, 0.5000, 0.5000, 0.5000,
         0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
         0.5000, 0.5000, 0.5000],
        [0.7311, 0.5000, 0.5000, 0.5000, 0.7299, 0.5000, 0.5000, 0.5000, 0.7310,
         0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5001, 0.5002, 0.5000,
         0.5000, 0.5000, 0.5000]], device='cuda:0')

In [55]:
df.iloc[-2]['generate']

'Laurence Siegel , từng pgụ trách mảng nghiên cứu tại F ord F oundation , cho rằng sự thăng trầm của quỹ thuộc Mindich cho thấy Harvard cần giữ khoảng cách hơn với những nhà quản lý có mối quan hệ với Trường Đại học .'

In [56]:
df.iloc[-2]['text']

'Laurence Siegel , từng phụ trách mảng nghiên cứu tại F ord F oundation , cho rằng sự thăng trầm của quỹ thuộc Mindich cho thấy Harvard cần giữ khoảng cách hơn với những nhà quản lý có mối quan hệ với Trường Đại học .'

In [51]:
torch.argmax(a, dim=-1)

tensor([[ 332,  322,  965,  648,  589, 1599,   26,   26,    9,    9,    9,    9,
            9,   26,   26,   26,    9,    9,    9,   26,   26],
        [  11,  418,  114, 1993, 2336,   12,   80, 1965,   74,  812,  464,  261,
           48, 2063,   90,   60,  229,   29,  552, 1201,  111]],
       device='cuda:0')

In [50]:
torch.sigmoid(b)

tensor([[0.5017, 0.5006, 0.5002, 0.5006, 0.6974, 0.5000, 0.5000, 0.5000, 0.5000,
         0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
         0.5000, 0.5000, 0.5000],
        [0.5002, 0.5045, 0.5026, 0.5000, 0.5027, 0.5009, 0.5004, 0.5003, 0.7283,
         0.5020, 0.5010, 0.5003, 0.5011, 0.5001, 0.6719, 0.5002, 0.7303, 0.5025,
         0.5030, 0.5023, 0.5000]], device='cuda:0')

In [54]:
reverse_map_word[90]

'thực'

In [43]:
import torch

spell_model = SpellCorrectionModel(13130,214).to(device)
spell_model.load_state_dict(torch.load("model_loss_1.8290443115711212.pth")['model_state_dict'])

<All keys matched successfully>

In [53]:
g = -196
input_sen = df['generate'].iloc[g]
label = df['text'].iloc[g]
w, c, max_length = mapping_batch(["tét",input_sen])
spell_model.eval()
with torch.no_grad():
    a, b, c, d = spell_model(w,c)
correction = torch.argmax(a, dim=-1).tolist()[1]
spell_output = torch.sigmoid(b).tolist()[1]
spell_output = [1 if i > 0.7 else 0 for i in spell_output ]
upper_output = torch.sigmoid(b).tolist()[1]
upper_output = [1 if i > 0.7 else 0 for i in upper_output ]
res = []
for i, j, k in zip(input_sen.split(' '), spell_output, correction):
    if j == 1:
        res.append(reverse_map_word[k])
    else:
        res.append(i)
print("Input : ",input_sen)
print("Groundtruth : ",label)
print("Fix : ",' '.join(x for x in res))
        


Input :  Mô5t vai vị khách còn không kìm được nước mắt .
Groundtruth :  Một vài vị khách còn không kìm được nước mắt .
Fix :  một vai vị khách còn không kìm được nước mắt .


In [54]:
df.iloc[g]

text            Một vài vị khách còn không kìm được nước mắt .
generate       Mô5t vai vị khách còn không kìm được nước mắt .
spell_label                  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
cap_label                    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
word_count                                                  11
Name: 999802, dtype: object

In [60]:
tmp_data = [df.iloc[g]]
batch_input_gt = [x['text'] for x in tmp_data]
batch_input_gt
label_correction = [get_correct(x['text'], x['generate']) for x in tmp_data]
label_correction = [expand_to_max_length(x, 11) for x in label_correction]

In [61]:
label_correction

[[16, 663, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [65]:
reverse_map_word[2]

'<pad>'

In [45]:
out

tensor([[[ 1.3997e+00,  1.9536e+00, -2.3226e-01, -1.3761e+00,  3.3113e-02,
          -5.7060e-01, -1.7402e-01,  5.9783e-01, -1.1561e+00, -4.7511e-01],
         [ 1.0332e+00,  1.7896e+00, -5.2333e-01, -1.1759e+00, -2.9186e-01,
           2.2583e-03,  1.0198e-01,  1.2120e+00, -7.5648e-01, -1.3915e+00]],

        [[-2.4668e-01,  1.6173e+00, -9.8781e-01, -9.7483e-01,  3.7100e-01,
           3.5459e-01,  5.2244e-01,  1.4033e+00, -4.0863e-01, -1.6507e+00],
         [ 1.1081e+00,  1.5793e+00, -6.1418e-03, -1.4710e+00, -7.1274e-01,
          -9.2333e-01, -1.9583e-01,  1.5204e+00, -4.5299e-01, -4.4565e-01]],

        [[ 1.8982e-01,  1.5839e+00, -3.6538e-01, -1.6380e+00,  1.2425e+00,
          -6.6418e-01, -1.0880e+00,  9.9658e-01, -5.9971e-01,  3.4244e-01],
         [-1.2084e+00, -4.6442e-01,  2.5157e-01, -5.5118e-01,  1.4654e+00,
           5.9568e-02, -1.9100e+00,  4.3064e-01,  7.2227e-01,  1.2045e+00]],

        [[ 4.9238e-01, -7.4072e-01,  3.0203e-01, -1.6311e+00, -1.2081e+00,
           4.

/tmp/ipykernel_171466/1088423724.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:486.)
  out.grad
